# 1. Test di ipotesi a varianza nota

Le specifiche tecniche per la velocità di combustione di un propellente richiedono che deve essere di 50 cm/s. Sappiamo che la deviazione standard è di 2 cm/s. Si effettuano 24 misurazioni
51, 50.2, 49.5, 48.7, 50.2, 50.5, 49.6, 51.1, 50.6, 49.1, 53.1, 50.4, 49.3, 48.9, 50.3, 51.8, 51.3, 48.5, 49.3, 55.1, 53.1, 52.5, 55.1, 50.6
- Si può rigettare l’ipotesi nulla con un livello di significatività del 5%?
- E se invece si richiedesse l’1%?
- Calcolare infine il p-value.

Supponiamo che lo sperimentatore voglia impostare il test in modo che la reale velocità di combustione media differisca da 50 cm/s per al più 1 cm/s.<br> Si vuole inoltre che il test affermerà questo fatto (cioè rigetterà H0 : μ = 50) con una probabilità del 90% e un livello di significatività del 5%.
- Determinare la dimensione campionaria.

In [1]:
import numpy as np
from scipy.stats import norm, t, chi2, binom, poisson, rankdata

In [2]:
# Test bilatero
# H0 -> mu = mu_0
# H1 -> mu != mu_0

x = np.array([51, 50.2, 49.5, 48.7, 50.2, 50.5, 49.6, 51.1, 50.6, 49.1, 53.1, 50.4, 
              49.3, 48.9, 50.3, 51.8, 51.3, 48.5,49.3, 55.1, 53.1, 52.5, 55.1, 50.6])
sig = 2
mu_0 = 50
alpha = 0.05

n = x.size
mu = np.mean(x)
print(f'Media campionaria: {mu:.3f}')

# PAG 12
Z0 = (mu-mu_0)/sig * np.sqrt(n)
print(f'Z0: {Z0:.3f}')

# Test con livello di significativià al 5%
PHI = norm.ppf(1-alpha/2)
print(f'Phi: {PHI:.3f}')

cond = np.abs(Z0) > PHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")

# poichè|Z0| > phi si rigetta l'ipotesi nulla
# quindi si afferma che il propellente non rispetta le specifiche con una significatività del 5%

Media campionaria: 50.825
Z0: 2.021
Phi: 1.960
Rigetto l'ipotesi nulla


In [3]:
# Test con livello di significativià al 1%
print(f'Z0: {Z0:.3f}')

alpha = 0.01
PHI = norm.ppf(1-alpha/2)
print(f'Phi: {PHI:.3f}')

cond = np.abs(Z0) > PHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")

# poichè |Z0| < PHI non abbiamo sufficienti elementi per rigettare l'ipotesi nulla

Z0: 2.021
Phi: 2.576
Non ho elementi sufficienti per rigettare l'ipotesi nulla


In [4]:
# p-value : il più piccolo valore di alpha per il quale è possibile rigettare l'ipotesi

# Pag 14
p = 2 * (1 - norm.cdf(np.abs(Z0)))
print(f'P-value: {p:.3f}\n')

P-value: 0.043



In [5]:
# Calcoliamo la dimensione campionaria tramite errore di secondo tipo

alpha = 0.05
beta = 1 - 90/100 # probabilità di non rifiutare H0 quando è falsa
delta = 1 # quanto ci stacchiamo dalla media vera

PHI_alpha = norm.ppf(1-alpha/2)
print(f'Phi: {PHI_alpha:.3f}')

# Pag 16
check = -PHI_alpha-delta*np.sqrt(n)/sig # se <= -3 approssima una normale
print(f'Vale {check:.3f} <= -3, quindi la distribuzione approssima una normale') 

PHI_beta = norm.ppf(1-beta)
dim_camp = (PHI_alpha+PHI_beta)**2 * (sig**2/delta**2)
print(f"Dimensione del campione: {int(dim_camp)}")

Phi: 1.960
Vale -4.409 <= -3, quindi la distribuzione approssima una normale
Dimensione del campione: 42


# 2. Test sulla media con varianza ignota

Si vuole testare ad un livello di significatività α = 0.05 se il carico di rottura di un materiale supera 10 MPa, tenendo presente che 22 prove hanno fornito i seguenti risultati
19.8 18.5 17.6 16.7 15.8 15.4 14.1 13.6 11.9 11.4 11.4 8.8 7.5 15.4 15.4 19.5 14.9 12.7 11.9 11.4 10.1 7.9
- Calcolare inoltre il p-value.

In [6]:
# Test unilatero a destra
# Ipotesi nulla         H_0 : mu = mu_0
# Ipotesi alternativa   H_1 : mu > mu_0

x = np.array([19.8, 18.5, 17.6, 16.7, 15.8, 15.4, 14.1, 13.6, 11.9, 11.4, 11.4,
              8.8, 7.5, 15.4, 15.4, 19.5, 14.9, 12.7, 11.9, 11.4, 10.1, 7.9,])

alpha = 0.05
mu_0 = 10

n = x.size
mu = np.mean(x)
print(f'Media campionaria: {mu:.3f}')
S = np.std(x, ddof=1)
print(f'Deviazione standard campionaria: {S:.3f}')

# Pag 19
T0 = (mu-mu_0)/S * np.sqrt(n)
print(f'T0: {T0:.3f}')

# Test unilatero a destra
T = t.ppf(1-alpha, n-1)
print(f't: {T:.3f}')

cond = T0 > T
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")
    
# T0 è maggiore di t pertanto si rigetta l'ipotesi nulla in favore dell'ipotesi alternativa
# affermando che il carico di rottura supera significativamente il valore di 10

Ft = t.cdf(T0, n-1)
p = 1-Ft
print(f'P-value: {p}')

Media campionaria: 13.714
Deviazione standard campionaria: 3.554
T0: 4.902
t: 1.721
Rigetto l'ipotesi nulla
P-value: 3.781272593450513e-05


# 3. Test sulla varianza

Un macchinario riempie automaticamente delle bottiglie. Da un campione di 20 misurazioni si ottengono i seguenti valori (in litri)
2.05, 2.04, 1.98, 1.96, 2.03, 2.01, 1.97, 1.99, 2.01, 2.05 1.96, 1.95, 2.04, 2.01, 1.97, 1.96, 2.02, 2.04, 1.98, 1.94
Se la deviazione standard fosse superiore a 0.05 litri, la proporzione di bottiglie sotto o sovrariempite sarebbe non accettabile.

- I dati del campione contengono prove che suggeriscono che il produttore abbia un problema con le bottiglie riempite troppo o troppo poco?<br>Utilizzare α = 0.05 e assumere che il volume di riempimento abbia una distribuzione normale.

In [7]:
# Test unilatero a sinistra
# HP0: sig^2 = sig0^2
# HP1: sig^2 < sig0^2 perchè voglio testare che sia accettabile (se fosse superiore non sarebbe accettabile)

x = np.array([2.05, 2.04, 1.98, 1.96, 2.03, 2.01, 1.97, 1.99, 2.01, 2.05, 1.96, 1.95, 2.04, 2.01, 1.97, 1.96, 2.02, 2.04, 1.98, 1.94])
n = x.size
sig0 = 0.05
alpha = 0.05

S = np.std(x, ddof=1)
S2 = S**2

print(f'Deviazione standard campionaria: {S:.3f}')
print(f'Varianza campionaria: {S2:.3f}')

# Pag 24
W0 = S2 / sig0**2 * (n-1)
print(f'W0: {W0:.3f}')

CHI = chi2.ppf(alpha, n-1)
print(f'CHI: {CHI:.3f}')

cond = W0 < CHI 
print("Rigetto l'ipotesi nulla" if cond else "Non ho elementi sufficienti per rigettare l'ipotesi nulla")
# Dato che W0 < CHI posso rigettare l'ipotesti nulla a favore dell'ipotesti alternativa, quindi la produzione sta andando bene

Deviazione standard campionaria: 0.036
Varianza campionaria: 0.001
W0: 9.968
CHI: 10.117
Rigetto l'ipotesi nulla


# 4. Test sulla proporzione (campione di leggi di Bernoulli)
In una catena di produzione si vuole mantenere il numero di pezzi difettosi al di sotto del 5%. Si analizza un campione di 200 pezzi e si trovano 4 pezzi difettosi.
- Si può asserire ad un livello di significativita α = 0.05 che la produzione rispetta le aspettative?
- Supponendo che il valore vero sia p∗ = 0.03 e supponendo che il costruttore voglia accettare un valore dell’errore di secondo tipo β = 0.1, quale ampiezza dovrebbe avere il campione?

In [8]:
# Test unilatero a sinistra
# H0 : p = p0
# H1 : p < p0

p0 = 5/100
n = 200
k = 4   # numero di successi --> numero di pezzi difettosi
alpha = 0.05

# Pag 26
p = k/n  # media campionaria
print(f'Media campionaria: {p}')

Z0 = (p-p0) / (np.sqrt( p0*(1-p0) )) * np.sqrt(n)
print(f'Z0: {Z0:.3f}')

PHI_alpha = norm.ppf(alpha)
print(f'PHI: {PHI_alpha:.3f}')

cond = Z0 < PHI_alpha
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")
# dato che Z0 < PHI si rigetta l'ipotesi nulla e si avalla l'ipotesi alternativa quindi le aspettative di produzione sono rispettate

Media campionaria: 0.02
Z0: -1.947
PHI: -1.645
Rigetto l'ipotesi nulla


In [9]:
p_star = 0.03 # p0 + delta
beta = 0.1

PHI_beta = norm.ppf(beta)
print(f'PHI_beta: {PHI_beta:.3f}')

dim_campionaria = ((PHI_beta * np.sqrt(p_star*(1-p_star))) + (PHI_alpha * np.sqrt(p0*(1-p0))) / (p0-p_star) )**2
print(f'Dimensione campionaria: {int(dim_campionaria)}')

PHI_beta: -1.282
Dimensione campionaria: 329


# 5. Test sulla media per coppie di popolazioni

Si vogliono confrontare due tipi di preparati per pittura. Ci si aspetta un diverso tempo di essiccamento. Si può supporre che la deviazione standard del tempo di essiccamento per ciascun tipo di essiccamento sia 8 minuti. 10 pareti vengono tinteggiate con il trattamento 1 e altrettamente pareti con il trattamento 2. <br>Si rilevano le medie campionare X_bar = 121 minuti e Y_bar = 112 minuti. 
- Si può trarre la clusione che il tempo di essiccamento del campione 1 sia maggiore di quello del campione 2 assumendo α = 0.05?
- Calcolare l’intervallo di confidenza per la differenza dei tempi medi di essiccamento.

In [10]:
# Test unilatero a destra
# H0: mu1 = mu2 
# H1: mu1 > mu2 

sig_1 = sig_2 = 8   # Deviazione standard comune per entrambi i trattamenti
n = m = 10     

mu_1 = 121  
mu_2 = 112 

alpha = 0.05 

# Pag 31
sig2_1 = sig2_2 = sig_1**2
Z0 = (mu_1-mu_2)/np.sqrt((sig2_1/n) + (sig2_2/m))
print(f'Z0: {Z0:.3f}')

PHI = norm.ppf(1-alpha)
print(f'PHI: {PHI:.3f}')

cond = Z0 > PHI
msg = "Si può trarre la clusione che il tempo di essiccamento del campione 1 sia maggiore di quello del campione 2 assumendo α = 0.05\n"
print(f"\nRigetto l'ipotesi nulla\n{msg}" if cond else "\nNon ho abbastanza elementi per rigettare l'ipotesi nulla\n")

# Se si vuole calolare il p-value
p_value = 1 - norm.cdf(Z0)
print(f'P-value: {p_value:.3f}')

# Calcolo dell'intervallo di confidenza per la differenza delle medie
A = mu_1-mu_2-PHI*np.sqrt((sig2_1/n) + (sig2_2/m))
B = "+ inf"

msg = f'''\nVale a dire che il tempo di essiccamento del primo preparato per pittura è di almeno {round(A, 2)} minuti
maggiore di quello del secondo preparato per pittura con una significatività del 95%.'''

print(f'\nIntervallo di confidenza: ] {round(A,2)}, {B} [\n{msg}')

Z0: 2.516
PHI: 1.645

Rigetto l'ipotesi nulla
Si può trarre la clusione che il tempo di essiccamento del campione 1 sia maggiore di quello del campione 2 assumendo α = 0.05

P-value: 0.006

Intervallo di confidenza: ] 3.12, + inf [

Vale a dire che il tempo di essiccamento del primo preparato per pittura è di almeno 3.12 minuti
maggiore di quello del secondo preparato per pittura con una significatività del 95%.


# 6. Test t per coppie di dati

Quindici adulti di età compresa tra 35 e 50 anni parecipano ad uno studio per valutare gli effetti di dieta alimentare ed esercizio fisico sul livello di colesterolo nel sangue. In ogni individuo il livello di colesterolo è stato misurato inizialmente e tre mesi dopo la dieta e l’allenamento. 
- Con un livello di significatività α = 0.05, è possibile avallare l’ipotesi che dieta ed esercizio fisico portino a ridurre il livello medio di colesterolo?

In [11]:
# Test unilatero a destra
# H0 -> mu_D = 0
# H1 -> mu_D > 0

x_before = np.array([265, 240, 258, 295, 251, 254, 287, 314, 260, 279, 283, 240, 238, 225, 247])
x_after = np.array([229, 231, 227, 240, 238, 241, 234, 256, 247, 239, 246, 218, 219, 226, 233])
alpha = 0.05

D = x_before - x_after
n = D.size
print(f'D: {D}')

D_bar = np.mean(D)
print(f'D_bar: {D_bar:.2f}')

S = np.std(D, ddof=1)
print(f'S: {S:.3f}')

T0 = D_bar/S * np.sqrt(n)
print(f'T0: {T0:.3f}')

T = t.ppf(1-alpha, n-1)
print(f'T: {T:.3f}')

cond = T0 > T
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")
# Rigetto l'ipotesi nulla in favore della prova alternativa, quindi il trattamento da risultati significativi

D: [36  9 31 55 13 13 53 58 13 40 37 22 19 -1 14]
D_bar: 27.47
S: 18.396
T0: 5.783
T: 1.761
Rigetto l'ipotesi nulla


# 7. Test del chi-quadro (campione di leggi Multinomiali)

Si vuole testare se un dado sia equilibrato o meno ad un livello di significatività α = 0.05. Si effettuano 100 lanci, registrando i risultati riportati nella seguente tabella.

20, 7, 12, 18, 20, 23

Dopo aver aumentato la dimensione campionaria, si registrano i risultati riportati nella seguente tabella. 
- Si ripeta il test di cui sopra con i nuovi dati.

388, 322, 314, 316, 344, 316

- Si ripeta quest’ultimo test con α = 0.01.

In [12]:
# H0 -> Dado equilibrato
# H1 -> Dado non equilibrato

x = np.array([20, 7, 12, 18, 20, 23])
m = x.size
n = sum(x) # 100

p = 1/6

# Pag 40
E = np.ones(m) * n * p # array di m elementi n*p
print(f'Frequenze assolute teoriche: {np.round(E, 3)}')

Tn = sum((x-E)**2 / E) # frequenze assolute
print(f'Tn: {Tn:.3f}')

alpha = 0.05
CHI = chi2.ppf(1-alpha, m-1)
print(f'CHI: {CHI:.3f}')

cond = Tn > CHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")

Frequenze assolute teoriche: [16.667 16.667 16.667 16.667 16.667 16.667]
Tn: 10.760
CHI: 11.070
Non ho abbastanza elementi per rigettare l'ipotesi nulla


In [13]:
x = np.array([388, 322, 314, 316, 344, 316])
n = np.sum(x) # 2000

E = np.ones(m) *p *n # array di 6 elementi n*p
print(f'Frequenze assolute attese: {np.round(E,3)}')

Tn = sum((x-E)**2 / E)
print(f'Tn: {Tn:.3f}')

cond = Tn > CHI
print(f"Se n = {n} allora", end=" ")
print("rigetto l'ipotesi nulla" if cond else "non ho abbastanza elementi per rigettare l'ipotesi nulla")
# Dado non equilibrato

Frequenze assolute attese: [333.333 333.333 333.333 333.333 333.333 333.333]
Tn: 12.616
Se n = 2000 allora rigetto l'ipotesi nulla


In [14]:
alpha = 0.01
CHI = chi2.ppf(1-alpha, m-1)
print(f'CHI: {CHI:.3f}')

cond = Tn > CHI
print(f"Se alpha = {alpha} allora", end=" ")
print("rigetto l'ipotesi nulla" if cond else "non ho abbastanza elementi per rigettare l'ipotesi nulla")

CHI: 15.086
Se alpha = 0.01 allora non ho abbastanza elementi per rigettare l'ipotesi nulla


# 8. Test di adattamento (ricondotto al chi-quadro)

Si può adattare una distibuzione di Poisson ai dati della seguente tabella ? 584, 398, 165, 35, 15

In [15]:
# H0 -> I dati seguono una distribuzione di Poisson
# H1 -> I dati NON seguono una distribuzione di Poisson

x = np.array([584, 398, 165, 35, 15])
m = x.size
n = np.sum(x)

pk_bar = x/n 
print(f'Frequenze relative empiriche: {pk_bar}')

# Stimo lambda con la media campionaria
Lambda = np.sum(x * np.array([0, 1, 2, 3, 4]))/n
print(f'Lambda: {Lambda:.3f}')

pk = np.zeros(m)
for i in range(m-1):
    pk[i] = poisson.pmf(i, Lambda)   # i -> numero di successi
pk[m-1] = 1-np.sum(pk)
print(f'Frequenze relative teoriche: {pk}')

# Teorema di Pearson Pag 38
Tn = n*np.sum((pk_bar-pk)**2 / pk) # frequenze relative
print(f'Tn: {Tn:.3f}')

alpha = 0.05
r = 1 # numero di parametri stimati -> Lambda
CHI = chi2.ppf(1-alpha, m-r-1)
print(f'CHI: {CHI:.3f}')

cond = Tn > CHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")

Frequenze relative empiriche: [0.48788638 0.33249791 0.13784461 0.02923977 0.01253133]
Lambda: 0.746
Frequenze relative teoriche: [0.47424475 0.35380164 0.13197363 0.03281884 0.00716115]
Tn: 7.605
CHI: 7.815
Non ho abbastanza elementi per rigettare l'ipotesi nulla


# 9. Test del chi-quadro di indipendenza

Si vuole testare se un antibiotico è efficace. Si considerano 170 pazienti. I dati ottenuti sono stati raccolti nella tabella seguente, detta tabella di contingenza:

- I due effetti, trattamento e guarigione, sono indipendenti ?

In [16]:
x = np.array([[44, 10], [81, 35]])
n = 170
m = r = 2 # possibili esiti per X e per Y
alpha = 0.05

# pag 48
# Frequenze relative empiriche di trattamento
p_bar = np.array([54, 116])/n
print(f'Frequenze relative relative di trattamento: {p_bar}')

# Frequenze relative empiriche di guarigione
q_bar = np.array([125, 45])/n
print(f'Frequenze relative relative di guarigione: {q_bar}')

# Probabilità congiunte empiriche
pi_greco = x/n
print(f'Probabilità congiunte empiriche: \n{pi_greco}\n')

Tn = 0 
for h in range(m):
    for k in range(r):
        Tn += n* ((p_bar[h]*q_bar[k] - pi_greco[h][k])**2 / pi_greco[h][k])
print(f'Tn: {Tn:.3f}')

CHI = chi2.ppf(1-alpha, (m-1)*(r-1))
print(f'CHI: {CHI:.3f}')

cond = Tn > CHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")

Frequenze relative relative di trattamento: [0.31764706 0.68235294]
Frequenze relative relative di guarigione: [0.73529412 0.26470588]
Probabilità congiunte empiriche: 
[[0.25882353 0.05882353]
 [0.47647059 0.20588235]]

Tn: 3.018
CHI: 3.841
Non ho abbastanza elementi per rigettare l'ipotesi nulla


# **Test non parametrici**

# 10. Test della mediana

Un motore a reazione è formato legando insieme un propellente di accensione e un propellente di sostegno all’interno di un alloggiamento metallico. <br>La resistenza al taglio del legame tra i due tipi di propellente è una caratteristica importante.
- Vogliamo testare l’ipotesi che la mediana della resistenza al taglio sia 2000 psi con una significatività α = 0.05.

I dati sono riportati nel file Dataset motore.dat.

In [21]:
# Test bilatero
# H0 -> mu = 2000
# H1 -> mu != 2000

x = np.loadtxt("Dataset_motore.dat")
n = x.size
mu = 2000
p = 1/2
alpha = 0.05

D = x - mu
# print(f'Differenze: {D}')

r = sum((D>0))
print(f'Numero di diff. positive: {r}')
# poichè r > n/2  si ha che p_value = 2P(R >= r)

p_value = 2 * sum(binom.pmf(k, n, p) for k in range(r, n+1)) # numeri da r a n
print(f'P-value: {p_value:.3f}')

cond = p_value <= alpha
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")

Numero di diff. positive: 14
P-value: 0.115
Non ho abbastanza elementi per rigettare l'ipotesi nulla


# 11. Test di Kruskal-Wallis

In un esperimento si confrontano quattro diverse tecniche di mescolamento per il cemento e si misura la resistenza alla trazione. 
- Si può affermare che la tecnica di mescolamento influisca sulla resistenza alla trazione? Si usi α = 0.05.

In [18]:
# H0 -> Tutte le mediane sono uguali, ovvero che la tecnica non influisce sulla trazione
# H1 -> La tecnica di mescolamento influisce sulla resistenza alla trazione

x1 = np.array([3129, 3000, 2865, 2890])
x2 = np.array([3200, 3000, 2975, 3150])
x3 = np.array([2800, 2900, 2985, 3050])
x4 = np.array([2600, 2700, 2600, 2765])

m = 4 # no. campioni
n = np.array([x1.size, x2.size, x3.size, x4.size]) 
N = np.sum(n)
print(f'Numero totale di osservazioni: {N}') 
alpha = 0.05

x = np.concatenate([x1, x2, x3, x4])
R = rankdata(x) # ranghi da 1 ad N
print(f'Ranghi: {R}')

# Siamo in presenza di dati ripetuti (Pag 62)
S2 = (np.sum(R**2) - (N*(N+1)**2) / 4) / (N-1)
print(f'S2: {S2}')

# Somma dei ranghi per ogni campione
RR = np.zeros(m)
for i in range(m):
    RR[i] = np.sum(R[m*i:m*(i+1)])
print(f'RR: {RR}')

H = (np.sum(RR**2/n) - N*(N+1)**2 /4)/S2
print(f'H: {H}')

CHI = chi2.ppf(alpha, m-1)
print(f'CHI: {CHI}')

cond = H >= CHI
print("Rigetto l'ipotesi nulla" if cond else "Non ho abbastanza elementi per rigettare l'ipotesi nulla")

Numero totale di osservazioni: 16
Ranghi: [14.  11.5  6.   7.  16.  11.5  9.  15.   5.   8.  10.  13.   1.5  3.
  1.5  4. ]
S2: 22.6
RR: [38.5 51.5 36.  10. ]
H: 10.027654867256636
CHI: 0.35184631774927144
Rigetto l'ipotesi nulla
